In [1]:
import finnhub
import time
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType, IntegerType
from pyspark.sql import Row
from pyspark.sql.functions import mean
from pyspark.sql.functions import date_format
import datetime

In [2]:
spark = SparkSession.builder.appName("StockData").getOrCreate()
finnhub_client = finnhub.Client(api_key="ckmaplhr01qu6et565b0ckmaplhr01qu6et565bg")
# define datafrme
schema = StructType([
    StructField("Stock Name", StringType(), True),
    StructField("UTC Timestamp", TimestampType(), True),
    StructField("c", DoubleType(), True),
    StructField("l", DoubleType(), True),
    StructField("h", DoubleType(), True),
    StructField("o", DoubleType(), True),
    StructField("v", DoubleType(), True)
])
schema_MA = StructType([
    StructField("Stock", StringType(), True),
    StructField("Datetime", TimestampType(), True),
    StructField("c_MA", DoubleType(), True),
    StructField("l_MA", DoubleType(), True),
    StructField("h_MA", DoubleType(), True),
    StructField("o_MA", DoubleType(), True),
    StructField("v_MA", DoubleType(), True)
])


total_runtime = 30 * 60

api_call_interval = 5 * 60

total_api_calls = total_runtime // api_call_interval

end_time = int(time.time())
start_time = end_time - 3600

for m in range(total_api_calls):

    data = finnhub_client.stock_candles('AAPL', '1', start_time, end_time)
    stock_name = "APPL"  
    timestamps = data["t"]
    closes = data["c"]
    lows = data["l"]
    highs = data["h"]
    opens = data["o"]
    volumes = data["v"]
    rows=[]
    for i in range(len(timestamps)):
        row = Row("stock_name", "timestamp", "c", "l", "h", "o", "v")(stock_name,datetime.datetime.utcfromtimestamp(timestamps[i]), float(closes[i]), float(lows[i]), float(highs[i]), float(opens[i]), float(volumes[i]))
        rows.append(row)
        
    if m==0:
        df = spark.createDataFrame(rows, schema)
    else:
        df1=spark.createDataFrame(rows, schema)
        difer=df1.join(df,df1["UTC Timestamp"]==df["UTC Timestamp"], "left_anti")
        df=df.union(difer)
        
    sorted_df = df.orderBy("UTC Timestamp")
    sorted_df.show(10)
    lastrows = sorted_df.tail(10)
    lastdf=spark.createDataFrame(lastrows)
    lastdf.show()
    
    mean_values = df.agg(mean("c").alias("avg_c"),
                    mean("l").alias("avg_l"),
                    mean("h").alias("avg_h"),
                    mean("o").alias("avg_o"),
                    mean("v").alias("avg_v")).collect()[0]

    row_MA = Row(stock_name="APPL",
                 timestamp=datetime.datetime.utcfromtimestamp(end_time),
                 c=mean_values.avg_c,
                 l=mean_values.avg_l,
                 h=mean_values.avg_h,
                 o=mean_values.avg_o,
                 v=mean_values.avg_v)
    
    if m==0:
        df_MA = spark.createDataFrame([row_MA], schema_MA)
    else:
        df1_MA=spark.createDataFrame([row_MA], schema_MA)
        df_MA=df_MA.union(df1_MA)
    
    df_MA.show()
    time.sleep(api_call_interval)

    end_time = int(time.time())
    start_time = end_time - 3600


23/10/18 01:39:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+-------------------+--------+--------+--------+--------+--------+
|Stock Name|      UTC Timestamp|       c|       l|       h|       o|       v|
+----------+-------------------+--------+--------+--------+--------+--------+
|      APPL|2023-10-17 17:40:00| 177.645|177.5801|   177.7|  177.68| 57960.0|
|      APPL|2023-10-17 17:41:00|  177.65| 177.645|  177.71|  177.65| 46743.0|
|      APPL|2023-10-17 17:42:00|  177.65|  177.63|  177.71|  177.65| 54681.0|
|      APPL|2023-10-17 17:43:00|177.6199|177.5225|  177.66|  177.65|101147.0|
|      APPL|2023-10-17 17:44:00|177.7499|  177.61|177.7503|177.6199| 72581.0|
|      APPL|2023-10-17 17:45:00|177.7601|  177.72|177.7868|  177.75| 57735.0|
|      APPL|2023-10-17 17:46:00|  177.64| 177.635|  177.77|177.7603| 63343.0|
|      APPL|2023-10-17 17:47:00|  177.51|  177.48|  177.65|  177.64| 88912.0|
|      APPL|2023-10-17 17:48:00|  177.57| 177.481|  177.57|  177.51|132862.0|
|      APPL|2023-10-17 17:49:00|  177.54|   177.5|  177.58|177.5

+----------+-------------------+--------+--------+--------+--------+--------+
|Stock Name|      UTC Timestamp|       c|       l|       h|       o|       v|
+----------+-------------------+--------+--------+--------+--------+--------+
|      APPL|2023-10-17 17:40:00| 177.645|177.5801|   177.7|  177.68| 57960.0|
|      APPL|2023-10-17 17:41:00|  177.65| 177.645|  177.71|  177.65| 46743.0|
|      APPL|2023-10-17 17:42:00|  177.65|  177.63|  177.71|  177.65| 54681.0|
|      APPL|2023-10-17 17:43:00|177.6199|177.5225|  177.66|  177.65|101147.0|
|      APPL|2023-10-17 17:44:00|177.7499|  177.61|177.7503|177.6199| 72581.0|
|      APPL|2023-10-17 17:45:00|177.7601|  177.72|177.7868|  177.75| 57735.0|
|      APPL|2023-10-17 17:46:00|  177.64| 177.635|  177.77|177.7603| 63343.0|
|      APPL|2023-10-17 17:47:00|  177.51|  177.48|  177.65|  177.64| 88912.0|
|      APPL|2023-10-17 17:48:00|  177.57| 177.481|  177.57|  177.51|132862.0|
|      APPL|2023-10-17 17:49:00|  177.54|   177.5|  177.58|177.5

+----------+-------------------+--------+--------+--------+--------+--------+
|Stock Name|      UTC Timestamp|       c|       l|       h|       o|       v|
+----------+-------------------+--------+--------+--------+--------+--------+
|      APPL|2023-10-17 17:40:00| 177.645|177.5801|   177.7|  177.68| 57960.0|
|      APPL|2023-10-17 17:41:00|  177.65| 177.645|  177.71|  177.65| 46743.0|
|      APPL|2023-10-17 17:42:00|  177.65|  177.63|  177.71|  177.65| 54681.0|
|      APPL|2023-10-17 17:43:00|177.6199|177.5225|  177.66|  177.65|101147.0|
|      APPL|2023-10-17 17:44:00|177.7499|  177.61|177.7503|177.6199| 72581.0|
|      APPL|2023-10-17 17:45:00|177.7601|  177.72|177.7868|  177.75| 57735.0|
|      APPL|2023-10-17 17:46:00|  177.64| 177.635|  177.77|177.7603| 63343.0|
|      APPL|2023-10-17 17:47:00|  177.51|  177.48|  177.65|  177.64| 88912.0|
|      APPL|2023-10-17 17:48:00|  177.57| 177.481|  177.57|  177.51|132862.0|
|      APPL|2023-10-17 17:49:00|  177.54|   177.5|  177.58|177.5

+----------+-------------------+--------+--------+--------+--------+--------+
|Stock Name|      UTC Timestamp|       c|       l|       h|       o|       v|
+----------+-------------------+--------+--------+--------+--------+--------+
|      APPL|2023-10-17 18:45:00|  177.04| 176.971| 177.128|  177.05| 73572.0|
|      APPL|2023-10-17 18:46:00|  177.05|177.0199|  177.08|  177.05| 44040.0|
|      APPL|2023-10-17 18:47:00|  177.09|  177.01|  177.14|  177.04|103628.0|
|      APPL|2023-10-17 18:48:00|  177.05|  177.02|  177.13|  177.08| 96503.0|
|      APPL|2023-10-17 18:49:00|177.0789|  177.03|  177.13|  177.06| 42903.0|
|      APPL|2023-10-17 18:50:00|  176.81|  176.56|  177.08|  177.08|501015.0|
|      APPL|2023-10-17 18:51:00|   176.8|176.7301| 176.825|   176.8|141101.0|
|      APPL|2023-10-17 18:52:00|  176.73|  176.64|176.8143| 176.809|107578.0|
|      APPL|2023-10-17 18:53:00| 176.755|  176.69|   176.8|176.7299| 78776.0|
|      APPL|2023-10-17 18:54:00|  176.49|  176.45|  176.76|  176

...

+----------+-------------------+-------+--------+--------+--------+--------+
|Stock Name|      UTC Timestamp|      c|       l|       h|       o|       v|
+----------+-------------------+-------+--------+--------+--------+--------+
|      APPL|2023-10-17 18:50:00| 176.81|  176.56|  177.08|  177.08|501015.0|
|      APPL|2023-10-17 18:51:00|  176.8|176.7301| 176.825|   176.8|141101.0|
|      APPL|2023-10-17 18:52:00| 176.73|  176.64|176.8143| 176.809|107578.0|
|      APPL|2023-10-17 18:53:00|176.755|  176.69|   176.8|176.7299| 78776.0|
|      APPL|2023-10-17 18:54:00| 176.49|  176.45|  176.76|  176.75|210169.0|
|      APPL|2023-10-17 18:55:00|176.455| 176.365|  176.48|  176.48|175835.0|
|      APPL|2023-10-17 18:56:00| 176.65|  176.46|  176.66|  176.46|155712.0|
|      APPL|2023-10-17 18:57:00| 176.58|176.5057|  176.66|  176.66|107299.0|
|      APPL|2023-10-17 18:58:00|  176.6|  176.52|  176.62| 176.565|123753.0|
|      APPL|2023-10-17 18:59:00| 176.62|  176.55|  176.67|  176.59| 78594.0|

+-----+-------------------+------------------+------------------+------------------+------------------+-----------------+
|Stock|           Datetime|              c_MA|              l_MA|              h_MA|              o_MA|             v_MA|
+-----+-------------------+------------------+------------------+------------------+------------------+-----------------+
| APPL|2023-10-17 18:39:07|177.31362500000003| 177.2449566666667|177.37991666666665|177.31804833333337|92189.33333333333|
| APPL|2023-10-17 18:44:17| 177.3010876923077|177.23510615384617|177.36849692307692| 177.3088907692308|90962.13846153846|
| APPL|2023-10-17 18:49:24| 177.2839942857143|177.21904000000004|177.35086142857142|177.29082714285718|89616.92857142857|
| APPL|2023-10-17 18:54:32|177.24619466666667|177.17870533333334|177.31786133333333| 177.2603573333334|97490.98666666666|
| APPL|2023-10-17 18:59:42|         177.20462|177.13504500000002|177.27411999999998|177.21602250000007|       99412.7125|
+-----+-----------------

+----------+-------------------+--------+--------+--------+--------+--------+
|Stock Name|      UTC Timestamp|       c|       l|       h|       o|       v|
+----------+-------------------+--------+--------+--------+--------+--------+
|      APPL|2023-10-17 17:40:00| 177.645|177.5801|   177.7|  177.68| 57960.0|
|      APPL|2023-10-17 17:41:00|  177.65| 177.645|  177.71|  177.65| 46743.0|
|      APPL|2023-10-17 17:42:00|  177.65|  177.63|  177.71|  177.65| 54681.0|
|      APPL|2023-10-17 17:43:00|177.6199|177.5225|  177.66|  177.65|101147.0|
|      APPL|2023-10-17 17:44:00|177.7499|  177.61|177.7503|177.6199| 72581.0|
|      APPL|2023-10-17 17:45:00|177.7601|  177.72|177.7868|  177.75| 57735.0|
|      APPL|2023-10-17 17:46:00|  177.64| 177.635|  177.77|177.7603| 63343.0|
|      APPL|2023-10-17 17:47:00|  177.51|  177.48|  177.65|  177.64| 88912.0|
|      APPL|2023-10-17 17:48:00|  177.57| 177.481|  177.57|  177.51|132862.0|
|      APPL|2023-10-17 17:49:00|  177.54|   177.5|  177.58|177.5

+----------+-------------------+--------+--------+--------+-------+--------+
|Stock Name|      UTC Timestamp|       c|       l|       h|      o|       v|
+----------+-------------------+--------+--------+--------+-------+--------+
|      APPL|2023-10-17 18:55:00| 176.455| 176.365|  176.48| 176.48|175835.0|
|      APPL|2023-10-17 18:56:00|  176.65|  176.46|  176.66| 176.46|155712.0|
|      APPL|2023-10-17 18:57:00|  176.58|176.5057|  176.66| 176.66|107299.0|
|      APPL|2023-10-17 18:58:00|   176.6|  176.52|  176.62|176.565|123753.0|
|      APPL|2023-10-17 18:59:00|  176.62|  176.55|  176.67| 176.59| 78594.0|
|      APPL|2023-10-17 19:00:00| 176.691|   176.6|  176.73| 176.62| 86674.0|
|      APPL|2023-10-17 19:01:00|  176.62| 176.575|176.7369|176.695| 83092.0|
|      APPL|2023-10-17 19:02:00|176.5948|  176.52|  176.63| 176.63| 86374.0|
|      APPL|2023-10-17 19:03:00| 176.625| 176.565|  176.65| 176.59| 74684.0|
|      APPL|2023-10-17 19:04:00|  176.68| 176.552|  176.69| 176.63| 91745.0|

+-----+-------------------+------------------+------------------+------------------+------------------+-----------------+
|Stock|           Datetime|              c_MA|              l_MA|              h_MA|              o_MA|             v_MA|
+-----+-------------------+------------------+------------------+------------------+------------------+-----------------+
| APPL|2023-10-17 18:39:07|177.31362500000003| 177.2449566666667|177.37991666666665|177.31804833333337|92189.33333333333|
| APPL|2023-10-17 18:44:17| 177.3010876923077|177.23510615384617|177.36849692307692| 177.3088907692308|90962.13846153846|
| APPL|2023-10-17 18:49:24| 177.2839942857143|177.21904000000004|177.35086142857142|177.29082714285718|89616.92857142857|
| APPL|2023-10-17 18:54:32|177.24619466666667|177.17870533333334|177.31786133333333| 177.2603573333334|97490.98666666666|
| APPL|2023-10-17 18:59:42|         177.20462|177.13504500000002|177.27411999999998|177.21602250000007|       99412.7125|
| APPL|2023-10-17 19:04:

In [ ]:





# switch data to rows
rows = []
for entry in response:
    stock_name = "APPL"  
    timestamps = entry["t"]
    closes = entry["c"]
    lows = entry["l"]
    highs = entry["h"]
    opens = entry["o"]
    volumes = entry["v"]
    for i in range(len(timestamps)):
        row = Row("stock_name", "timestamp", "c", "l", "h", "o", "v")(stock_name, timestamps[i], float(closes[i]), float(lows[i]), float(highs[i]), float(opens[i]), float(volumes[i]))
        rows.append(row)
        
unique_rows = list(set(rows))                                                                          
sorted_rows = sorted(unique_rows, key=lambda x: x.timestamp)
new_rows = [Row(stock_name=row.stock_name,
                timestamp=datetime.datetime.utcfromtimestamp(row.timestamp).strftime('%Y-%m-%d %H:%M:%S'),
                c=row.c,  
                l=row.l,
                h=row.h,
                o=row.o,
                v=row.v) for row in sorted_rows]



# import rows to dataframe
df = spark.createDataFrame(new_rows, schema)
df.show()

spark.stop()
